In [1]:
import os
import warnings

from db_queries import get_location_metadata
import pandas as pd

from covid_model_deaths.data import get_input_data, plot_crude_rates
from covid_model_deaths.globals import COLUMNS
from covid_model_deaths import runner

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99
warnings.simplefilter('ignore')

RUN_TYPE = 'dev'
DATA_VERSION = 'best'
DATESTAMP_LABEL = '2020_04_23_US_pipeline_rmb'

PEAK_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_20/peak_locs_april20_.csv.csv'
PEAK_DURATION_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/smooth_peak_duration.csv'
R0_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/R0_dates.csv'
LOCATION_SET_VERSION = 648


# # read in cov input file (predicted date of R0 == 1) to see if we are using these for a given location
# cov_df = pd.read_csv(R0_FILE)
# r0_locs = cov_df['location_id'].unique().tolist()
# del cov_df
r0_locs = []

# # less conservative peak ranges
# peak_dur_df = pd.read_csv(PEAK_DURATION_FILE)
# peak_dur_df = peak_dur_df.loc[peak_dur_df['Location'] != 'Colorado']
# peak_dur_df['peak start date'] = pd.to_datetime(peak_dur_df['peak start date'])
# peak_dur_df['peak end date'] = pd.to_datetime(peak_dur_df['peak end date'])

CODE_DIR = os.path.abspath('../src/covid_model_deaths')
OUTPUT_DIR = f'/ihme/covid-19/deaths/{RUN_TYPE}/{DATESTAMP_LABEL}'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
checkpoint = runner.Checkpoint(OUTPUT_DIR)


print(f'Writing to {OUTPUT_DIR}')
print(CODE_DIR)
print(checkpoint)

raw_draw_path = f'{OUTPUT_DIR}/euro_data.csv'
average_draw_path = f'{OUTPUT_DIR}/past_avg_euro_data.csv'
yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_22_US/state_data.csv'
before_yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_21_US/state_data.csv'


def filter_data(data: pd.DataFrame) -> pd.DataFrame:
    """Remove bad or problematic data locations and dates."""
    return data

def get_locations(location_set_version_id):
    loc_df = get_location_metadata(location_set_id=111, 
                                   location_set_version_id=location_set_version_id)
    most_detailed = loc_df['most_detailed'] == 1
    us = loc_df['path_to_top_parent'].str.startswith('102,')
    keep_columns = ['location_id', 'location_ascii_name', 'parent_id', 'level', 'most_detailed']
    
    us_df = loc_df.loc[most_detailed & us, keep_columns]
    us_df = us_df.rename(columns={'location_ascii_name': 'Location'})
    
    # Add parents
    loc_df = loc_df[['location_id', 'location_ascii_name']]
    loc_df = loc_df.rename(columns={'location_id':'parent_id',
                                    'location_ascii_name':'Country/Region'})
    
    us_df = us_df.merge(loc_df)
    us_df = us_df.loc[:, ['location_id', 'Location', 'Country/Region', 'level']]
    return us_df
    
    
    
    

Writing to /ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb
/ihme/code/rmbarber/covid-model-deaths/src/covid_model_deaths
Checkpoint(/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/checkpoint)


## read full (unrestricted) set from snapshot

In [2]:
loc_df = get_locations(LOCATION_SET_VERSION)
input_full_df = filter_data(get_input_data('full_data', DATA_VERSION))
input_death_df = filter_data(get_input_data('deaths', DATA_VERSION))
input_age_pop_df = get_input_data('age_pop', DATA_VERSION)
input_age_death_df = get_input_data('age_death', DATA_VERSION)

# Save pops for Bobby.
get_input_data('us_pops').to_csv(f'{OUTPUT_DIR}/pops.csv', index=False)

checkpoint.write('location', loc_df)
checkpoint.write('full_data', input_full_df)
checkpoint.write('deaths', input_death_df)
checkpoint.write('age_pop', input_age_pop_df)
checkpoint.write('age_death', input_age_death_df)

In [3]:
# death_df = checkpoint.load('deaths')
# plot
# plot_crude_rates(death_df, level='subnat')

## combine back-casted death rates with cases for abie (using model dataset, i.e. admin1 and below)

In [4]:
%%time
full_df = checkpoint.load('full_data')
death_df = checkpoint.load('deaths')
age_pop_df = checkpoint.load('age_pop')
age_death_df = checkpoint.load('age_death')

backcast_location_ids = runner.get_backcast_location_ids(full_df)
cases_and_backcast_deaths_df = runner.make_cases_and_backcast_deaths(full_df, death_df, age_pop_df, age_death_df, backcast_location_ids)

cases_and_backcast_deaths_df.to_csv(f'{OUTPUT_DIR}/backcast_for_case_to_death.csv', index=False)
checkpoint.write('cases_and_backcast_deaths', cases_and_backcast_deaths_df)

2020-04-24 13:52:47.277 | INFO     | covid_model_deaths.runner:load:47 - Loading full_data from in memory cache.
2020-04-24 13:52:47.279 | INFO     | covid_model_deaths.runner:load:47 - Loading deaths from in memory cache.
2020-04-24 13:52:47.280 | INFO     | covid_model_deaths.runner:load:47 - Loading age_pop from in memory cache.
2020-04-24 13:52:47.281 | INFO     | covid_model_deaths.runner:load:47 - Loading age_death from in memory cache.
100%|██████████| 178/178 [01:22<00:00,  2.16it/s]


CPU times: user 1.85 s, sys: 298 ms, total: 2.14 s
Wall time: 1min 23s


## Impute death thresholds.

In [5]:
%%time
cases_and_backcast_deaths_df = checkpoint.load('cases_and_backcast_deaths')
loc_df = checkpoint.load('location')
threshold_dates = runner.impute_death_threshold(cases_and_backcast_deaths_df,
                                                loc_df)
threshold_dates.to_csv(f'{OUTPUT_DIR}/threshold_dates.csv', index=False)
checkpoint.write('threshold_dates', threshold_dates)

2020-04-24 13:54:14.914 | INFO     | covid_model_deaths.runner:load:47 - Loading cases_and_backcast_deaths from in memory cache.
2020-04-24 13:54:14.916 | INFO     | covid_model_deaths.runner:load:47 - Loading location from in memory cache.
100%|██████████| 53/53 [00:02<00:00, 18.01it/s]


CPU times: user 964 ms, sys: 239 ms, total: 1.2 s
Wall time: 3.5 s


## Make last day data

In [6]:
full_df = checkpoint.load('full_data')
threshold_dates = checkpoint.load('threshold_dates')

date_mean_df = runner.make_date_mean_df(threshold_dates)
last_day_df = runner.make_last_day_df(full_df,date_mean_df)
last_day_df.to_csv(f'{OUTPUT_DIR}/last_day.csv', index=False)

checkpoint.write('date_mean', date_mean_df)
checkpoint.write('last_day', last_day_df)

2020-04-24 13:54:21.502 | INFO     | covid_model_deaths.runner:load:47 - Loading full_data from in memory cache.
2020-04-24 13:54:21.504 | INFO     | covid_model_deaths.runner:load:47 - Loading threshold_dates from in memory cache.


## Submit models

In [11]:
full_df = checkpoint.load('full_data')
death_df = checkpoint.load('deaths')
age_pop_df = checkpoint.load('age_pop')
age_death_df = checkpoint.load('age_death')
date_mean_df = checkpoint.load('date_mean')
last_day_df = checkpoint.load('last_day')
loc_df = checkpoint.load('location')

dcr_df, dhr_df, li_df = runner.make_leading_indicator(
    full_df.loc[full_df[COLUMNS.location_id].isin(loc_df[COLUMNS.location_id].to_list())]
)
dcr_df.to_csv(f'{OUTPUT_DIR}/lagged_death_to_case_ratios.csv', index=False)
dhr_df.to_csv(f'{OUTPUT_DIR}/lagged_death_to_hosp_ratios.csv', index=False)
li_df.to_csv(f'{OUTPUT_DIR}/leading_indicator.csv', index=False)
li_df = li_df[[COLUMNS.location_id, COLUMNS.date, COLUMNS.ln_age_death_rate]]
li_df = li_df.loc[~li_df[COLUMNS.ln_age_death_rate].isnull()]

submodel_dict = runner.submit_models(full_df, death_df, age_pop_df, age_death_df, date_mean_df, li_df,
                                     loc_df, r0_locs,
                                     PEAK_FILE, OUTPUT_DIR, DATA_VERSION, R0_FILE, CODE_DIR, verbose=True)

checkpoint.write('submodel_dict', submodel_dict)

2020-04-24 14:06:35.530 | INFO     | covid_model_deaths.runner:load:47 - Loading full_data from in memory cache.
2020-04-24 14:06:35.531 | INFO     | covid_model_deaths.runner:load:47 - Loading deaths from in memory cache.
2020-04-24 14:06:35.533 | INFO     | covid_model_deaths.runner:load:47 - Loading age_pop from in memory cache.
2020-04-24 14:06:35.534 | INFO     | covid_model_deaths.runner:load:47 - Loading age_death from in memory cache.
2020-04-24 14:06:35.535 | INFO     | covid_model_deaths.runner:load:47 - Loading date_mean from in memory cache.
2020-04-24 14:06:35.535 | INFO     | covid_model_deaths.runner:load:47 - Loading last_day from in memory cache.
2020-04-24 14:06:35.536 | INFO     | covid_model_deaths.runner:load:47 - Loading location from in memory cache.
  0%|          | 0/53 [00:00<?, ?it/s]

qsub -N curve_model_523_google_21 -P proj_covid -q d.q -b y -l m_mem_free=12G -l fthread=3 -o /share/temp/sgeoutput/covid/output/ -e /share/temp/sgeoutput/covid/errors/ /ihme/covid-19/miniconda/envs/covid-deaths-2020-04-24_13-45-44/bin/python /ihme/code/rmbarber/covid-model-deaths/src/covid_model_deaths/model.py --model_location_id 523 --data_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_google_21/523.csv" --cov_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_google_21/523_covariate.csv" --peaked_file "/ihme/covid-19/deaths/mobility_inputs/2020_04_20/peak_locs_april20_.csv.csv" --output_dir "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_google_21/523" --last_day_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/last_day.csv" --covariate_effect gamma --n_draws=333
Your job 40081769 ("curve_model_523_google_21") has been submitted

qsub -N curve_model_523_descartes_21 -P proj_covid -q d.q -b y -l m_mem_free=12G -l f

  2%|▏         | 1/53 [00:11<10:12, 11.77s/it]

qsub -N curve_model_523_safegraph_21 -P proj_covid -q d.q -b y -l m_mem_free=12G -l fthread=3 -o /share/temp/sgeoutput/covid/output/ -e /share/temp/sgeoutput/covid/errors/ /ihme/covid-19/miniconda/envs/covid-deaths-2020-04-24_13-45-44/bin/python /ihme/code/rmbarber/covid-model-deaths/src/covid_model_deaths/model.py --model_location_id 523 --data_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/523.csv" --cov_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/523_covariate.csv" --peaked_file "/ihme/covid-19/deaths/mobility_inputs/2020_04_20/peak_locs_april20_.csv.csv" --output_dir "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/523" --last_day_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/last_day.csv" --covariate_effect gamma --n_draws=334
Your job 40081772 ("curve_model_523_safegraph_21") has been submitted

qsub -N curve_model_524_google_21 -P proj_covid -q d.q -b y -l m_mem_f

  4%|▍         | 2/53 [00:21<09:31, 11.21s/it]

...

qsub -N curve_model_529_safegraph_21 -P proj_covid -q d.q -b y -l m_mem_free=12G -l fthread=3 -o /share/temp/sgeoutput/covid/output/ -e /share/temp/sgeoutput/covid/errors/ /ihme/covid-19/miniconda/envs/covid-deaths-2020-04-24_13-45-44/bin/python /ihme/code/rmbarber/covid-model-deaths/src/covid_model_deaths/model.py --model_location_id 529 --data_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/529.csv" --cov_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/529_covariate.csv" --peaked_file "/ihme/covid-19/deaths/mobility_inputs/2020_04_20/peak_locs_april20_.csv.csv" --output_dir "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/529" --last_day_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/last_day.csv" --covariate_effect gamma --n_draws=334
Your job 40081889 ("curve_model_529_safegraph_21") has been submitted

qsub -N curve_model_530_google_21 -P proj_covid -q d.q -b y -l m_mem_f

 15%|█▌        | 8/53 [01:22<07:40, 10.23s/it]

qsub -N curve_model_530_safegraph_21 -P proj_covid -q d.q -b y -l m_mem_free=12G -l fthread=3 -o /share/temp/sgeoutput/covid/output/ -e /share/temp/sgeoutput/covid/errors/ /ihme/covid-19/miniconda/envs/covid-deaths-2020-04-24_13-45-44/bin/python /ihme/code/rmbarber/covid-model-deaths/src/covid_model_deaths/model.py --model_location_id 530 --data_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/530.csv" --cov_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/530_covariate.csv" --peaked_file "/ihme/covid-19/deaths/mobility_inputs/2020_04_20/peak_locs_april20_.csv.csv" --output_dir "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/model_data_safegraph_21/530" --last_day_file "/ihme/covid-19/deaths/dev/2020_04_23_US_pipeline_rmb/last_day.csv" --covariate_effect gamma --n_draws=334
Your job 40081932 ("curve_model_530_safegraph_21") has been submitted



 15%|█▌        | 8/53 [01:24<07:52, 10.50s/it]


KeyboardInterrupt: 

## compile draws

In [ ]:
full_df = checkpoint.load('full_data')
age_pop_df = checkpoint.load('age_pop')
threshold_dates = checkpoint.load('threshold_dates')
submodel_dict = checkpoint.load('submodel_dict')
loc_df = checkpoint.load('location')

obs_df = full_df[full_df.location_id.isin(loc_df.location_id)]

draw_dfs, past_draw_dfs, models_used, days, ensemble_draws_dfs = runner.compile_draws(loc_df,
                                                                                      submodel_dict,
                                                                                      obs_df,
                                                                                      threshold_dates,
                                                                                      age_pop_df)

if 'location' not in models_used:
    raise ValueError('No location-specific draws used, must be using wrong tag')
draw_df = pd.concat(draw_dfs)
model_type_df = pd.DataFrame({'location': loc_df['Location'].tolist(),
                              'model_used': models_used})

# write
draw_df.to_csv(f'{OUTPUT_DIR}/state_data.csv', index=False)
model_type_df.to_csv(f'{OUTPUT_DIR}/state_models_used.csv', index=False)
ensemble_plot_path = runner.make_and_save_draw_plots(OUTPUT_DIR, loc_df,
                                                     ensemble_draws_dfs, days, models_used, age_pop_df)
print(ensemble_plot_path)
checkpoint.write('draw_data', draw_df)

## total US deaths in this run

In [ ]:
draw_df = checkpoint.load('draw_data')

runner.display_total_deaths(draw_df)

## combine with previous predictions

In [ ]:
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
raw_draw_path = f'{OUTPUT_DIR}/state_data.csv'
average_draw_path = f'{OUTPUT_DIR}/past_avg_state_data.csv'
yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_22_US/state_data.csv'
before_yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_21_US/state_data.csv'
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

avg_df = runner.average_draws(raw_draw_path, yesterday_draw_path, before_yesterday_draw_path)

## NO NEED TO DO THIS, FOR NOW ##
# avg_df = get_peak_date(past_draw_df, avg_df)

# store data
avg_df.to_csv(average_draw_path, index=False)
print(average_draw_path)

# plot
compare_average_plot_path = runner.make_and_save_compare_average_plots(OUTPUT_DIR,
                                                                       raw_draw_path,
                                                                       average_draw_path,
                                                                       yesterday_draw_path,
                                                                       before_yesterday_draw_path,
                                                                       'United States of America')
print(compare_average_plot_path)


In [ ]:
raw_draw_path = f'{OUTPUT_DIR}/state_data.csv'
previous_draw_path = '/ihme/covid-19/deaths/prod/2020_04_22_US/state_data.csv'

compare_to_previous_plot_path = runner.make_and_save_compare_to_previous_plots(OUTPUT_DIR, raw_draw_path, previous_draw_path, label='United States')


In [ ]:

viz_dir = runner.send_plots_to_diagnostics(DATESTAMP_LABEL,
                                           f'{OUTPUT_DIR}/ensemble_plot.pdf',
                                           compare_average_plot_path,
                                           compare_to_previous_plot_path)
print(viz_dir)

